In [1]:
import numpy 
numpy.random.seed(1337)

In [2]:
import h5py
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Input, MaxPooling2D, Dropout, AveragePooling2D, GlobalAveragePooling2D
from keras.layers import Conv2D, Embedding
from keras.optimizers import SGD, rmsprop
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder
from keras.layers.advanced_activations import LeakyReLU, PReLU, ELU
from keras_contrib.layers.advanced_activations import PELU, SReLU

Using TensorFlow backend.


In [3]:
batch_size=50
num_classes=1370
epochs=25

In [4]:
import os
from PIL import Image
import matplotlib.image as mpimg

In [5]:
os.chdir("E:");
#path="E:/Dataset_Final(March)/One";
path="E:/Dataset_Final(April)/Two";
classes=os.listdir(path)
x=[]#Datapoints 
y=[]#labels 
for fol in classes:
    imgfiles=os.listdir(path+u'\\'+fol);
    for img in imgfiles:
        im=mpimg.imread(path+u'\\'+fol+u'\\'+img);
        x.append(im)
        y.append(fol)
x=numpy.array(x)
y=numpy.array(y)

In [6]:
print(x.shape)
print(y.shape)
x=x.reshape((-1,100,100,1))

(3648, 100, 100)
(3648,)


In [7]:
n=x.shape[0]
print (n)
randomize=numpy.arange(n)
numpy.random.shuffle(randomize)
randomize
x=x[randomize]
y=y[randomize]

3648


In [8]:
test_split=round(n*2.7/3)
x_train=x[:test_split]
y_train=y[:test_split]
x_test=x[test_split:]
y_test=y[test_split:]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(3283, 100, 100, 1)
(3283,)
(365, 100, 100, 1)
(365,)


In [9]:
y_train=keras.utils.to_categorical(y_train, num_classes)
y_test=keras.utils.to_categorical(y_test, num_classes)
y_train[:1]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [10]:
inputs=Input(shape=(100,100,1))

# y=Conv2D(1,(3,3), padding='same')(inputs)
# z=keras.layers.add([inputs,y])

x=Conv2D(16, (10, 10), padding='same')(inputs)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)


x=Conv2D(32, (9, 9), padding='same')(x)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)


x=Conv2D(64, (8, 8), padding='same')(x)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)

x=Conv2D(128, (3, 3), padding='same')(x)
x=SReLU()(x)
x=MaxPooling2D(pool_size=(2,2))(x)

x=Dropout(0.25)(x)
x=Flatten()(x)
x=Dense(num_classes)(x)
output=Activation('softmax')(x)
model=Model([inputs], output)

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100, 100, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 16)      1616      
_________________________________________________________________
s_re_lu_1 (SReLU)            (None, 100, 100, 16)      640000    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 50, 50, 32)        41504     
_________________________________________________________________
s_re_lu_2 (SReLU)            (None, 50, 50, 32)        320000    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 32)        0         
__________

In [12]:
opt=keras.optimizers.RMSprop(lr=1e-3)

In [13]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [14]:
import h5py
from keras.callbacks import ModelCheckpoint
filepath='E:Dataset_Final(March)/Checkpoints/Level_2/Epoch_41/weights-improvement-{epoch:2d}-{val_acc:.2f}.hdf5'
checkpoint=ModelCheckpoint(filepath, monitor='val_acc', verbose=1, mode='max')

In [15]:
hist=model.fit(x_train, y_train, 
               batch_size=50,
               epochs=epochs,
               validation_data=(x_test, y_test),
               callbacks=[checkpoint]
               )

Train on 3283 samples, validate on 365 samples
Epoch 1/25
3283/3283 [==============================] - 321s 98ms/step - loss: 7.0800 - acc: 0.0024 - val_loss: 6.4285 - val_acc: 0.0164

Epoch 00001: saving model to E:Dataset_Final(March)/Checkpoints/Level_2/Epoch_41/weights-improvement- 1-0.02.hdf5
Epoch 2/25
3283/3283 [==============================] - 367s 112ms/step - loss: 4.4328 - acc: 0.1532 - val_loss: 3.6756 - val_acc: 0.1945

Epoch 00002: saving model to E:Dataset_Final(March)/Checkpoints/Level_2/Epoch_41/weights-improvement- 2-0.19.hdf5
Epoch 3/25
3283/3283 [==============================] - 389s 118ms/step - loss: 1.9893 - acc: 0.4596 - val_loss: 2.4481 - val_acc: 0.3753

Epoch 00003: saving model to E:Dataset_Final(March)/Checkpoints/Level_2/Epoch_41/weights-improvement- 3-0.38.hdf5
Epoch 4/25
3283/3283 [==============================] - 440s 134ms/step - loss: 1.1431 - acc: 0.6585 - val_loss: 1.5021 - val_acc: 0.6137

Epoch 00004: saving model to E:Dataset_Final(March)/Chec

In [ ]:
score=model.evaluate(x_test, y_test)

In [ ]:
score[1]

In [ ]:
y_pred=model.predict(x_test)# for dataset of 12 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

In [185]:
model.load_weights("E:Dataset_Final(March)/Checkpoints/Level_1/Epoch_41/weights-improvement-14-0.98.hdf5")# f1

In [186]:
score=model.evaluate(x_test, y_test)

51/51 [==============================] - 1s 11ms/step


In [187]:
score[1]

0.98039215686274506

In [42]:
y_pred=model.predict(x_test)# for dataset of 12 characters
print (y_pred)
y_pred=numpy.argmax(y_pred, axis=1)
print (y_pred)

[[  1.90199081e-17   2.09929721e-17   1.24922264e-17 ...,   8.88945090e-27
    3.39433694e-19   3.19130001e-27]
 [  4.16776558e-13   6.69391775e-14   1.73623789e-12 ...,   2.32223887e-24
    4.06781779e-14   3.80484654e-24]
 [  7.38239051e-19   2.34063017e-18   6.32201321e-19 ...,   3.94380360e-24
    3.25690139e-19   4.92971049e-21]
 ..., 
 [  1.82910345e-20   1.50606739e-19   1.21145241e-19 ...,   8.85208560e-29
    9.35655495e-21   3.05209556e-32]
 [  4.91960693e-20   2.13653046e-20   4.30499203e-20 ...,   1.54109752e-23
    3.21556899e-20   1.41870780e-23]
 [  1.43401566e-20   9.62631955e-20   7.26449789e-20 ...,   2.05515214e-27
    1.25837056e-20   7.37339895e-23]]
[1104 1094  867  641  995 1020 1055  935  963  177  351  124  178  305  791
  290  364  936  111  948  116  662  249  843  824  713  228  353  285  301
 1347  671   93 1115 1080  830 1295  656  859  842  837 1001   91  191 1121
  809  131  308  631  219 1096  161   41  657 1294 1010 1042 1285  782  702
  694 1131  105 

In [43]:
print(confusion_matrix(numpy.argmax(y_test, axis=1),y_pred))

[[1 0 0 ..., 0 0 0]
 [0 2 0 ..., 0 0 0]
 [0 0 2 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 1 0 0]
 [0 0 0 ..., 0 2 0]
 [0 0 0 ..., 0 0 1]]


In [44]:

print (classification_report(numpy.argmax(y_test, axis=1), y_pred))

             precision    recall  f1-score   support

         38       1.00      1.00      1.00         1
         40       1.00      1.00      1.00         2
         41       0.67      1.00      0.80         2
         42       0.00      0.00      0.00         1
         54       1.00      1.00      1.00         1
         59       1.00      1.00      1.00         1
         64       1.00      1.00      1.00         1
         65       1.00      1.00      1.00         1
         66       1.00      1.00      1.00         1
         67       1.00      0.50      0.67         2
         68       1.00      0.50      0.67         2
         71       1.00      1.00      1.00         1
         73       1.00      1.00      1.00         1
         74       1.00      0.50      0.67         2
         75       1.00      1.00      1.00         1
         78       1.00      1.00      1.00         1
         91       1.00      1.00      1.00         1
         92       1.00      1.00      1.00   

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [16]:
import pandas

pandas.DataFrame(hist.history).to_csv("E:Dataset_Final(March)/Checkpoints/Level_2/Epoch_41/Figure/History-970.csv")        
# visualizing losses and accuracy
train_loss=hist.history['loss']
val_loss=hist.history['val_loss']
train_acc=hist.history['acc']
val_acc=hist.history['val_acc']
xc=range(epochs)

In [17]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages

pp=PdfPages("E:Dataset_Final(March)/Checkpoints/Level_2/Epoch_41/Figure/Loss-970.pdf")
plt.figure(1,figsize=(7,5))
plt.plot(xc,train_loss)
plt.plot(xc,val_loss)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('train_loss vs val_loss')
plt.grid(True)
plt.legend(['train','val'])
print (plt.style.available)
plt.style.use(['classic'])
plt.savefig(pp, format='pdf')
pp.close()

['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', '_classic_test']


In [18]:
pp=PdfPages("E:Dataset_Final(March)/Checkpoints/Level_2/Epoch_41/Figure/Accuracy-970.pdf")
plt.figure(2,figsize=(7,5))
plt.plot(xc,train_acc)
plt.plot(xc,val_acc)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('train_acc vs val_acc')
plt.grid(True)
plt.legend(['train','val'],loc=4)
#print plt.style.available # use bmh, classic,ggplot for big pictures
plt.style.use(['bmh'])
plt.savefig(pp, format='pdf')
pp.close()